# Collect Data for Test 1
- YFinance Data
- Weekdays (int)
- Month (int)
- Trend (int)
- Buy or Sell

In [ ]:
import pandas as pd
from datetime import datetime
from data_collector import get_finance_data, determine_trend

## Training Data

In [ ]:
df = get_finance_data(symbols="^GDAXI", interval="1d",start=datetime(2015, 1, 1), end=datetime(2022, 12, 31))

df['trend'] = df.apply(determine_trend, axis=1)

# Initialize the 'Invest' column
df['Invest'] = 0

for index, row in df.iterrows():
    if index > 0: 
        if df.at[index, "trend"] == 1:
            df.at[index-1, "Invest"] = True
        else:
            df.at[index-1, "Invest"] = False
            
df["month"] = df["month"].astype(int)
df["Invest"] = df["Invest"].astype(bool)
df.to_pickle("../Data/train_dax_data.pkl")
display(df)

## Test Data

In [ ]:
df = get_finance_data(symbols="^GDAXI", interval="1d", start=datetime(2023, 1, 1), end=datetime(2023, 12, 31))

df['trend'] = df.apply(determine_trend, axis=1)

# Initialize the 'Invest' column
df['Invest'] = 0

for index, row in df.iterrows():
    if index > 0: 
        if df.at[index, "trend"] == 1:
            df.at[index-1, "Invest"] = True
        else:
            df.at[index-1, "Invest"] = False

df["month"] = df["month"].astype(int)
df["Invest"] = df["Invest"].astype(bool)
df.to_pickle("../Data/test_dax_data.pkl")
display(df)